In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:98% !important; }</style>")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# GIS imports
from shapely.geometry import Polygon
from opera_coverage import *
import geopandas as gpd
from shapely.geometry import box, Point
from rasterio.crs import CRS

# Misc imports
import numpy as np
from itertools import product
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import concurrent.futures
from typing import List

In [4]:
output_path = Path('../outputs/')
output_path.mkdir(exist_ok=True)

In [5]:
# helper functions

def get_cadence_df(geo:Polygon, daterange:List[datetime]):
    return get_coverage(['sentinel1','sentinel2','landsat8'], geo, daterange)

def get_lat(geom):
    return int(np.floor(geom.centroid.coords.xy[1][0]))

def get_lon(geom):
    return int(np.floor(geom.centroid.coords.xy[0][0]))

def make_box(row):
    return box(row['lon'], row['lat'], row['lon'] + 1, row['lat'] + 1)

In [6]:
def make_grid(x_start, y_start, x_end, y_end, X_SPACING = 1, Y_SPACING = 0.25, RADIUS = 0.1):
    lon_lat = list(product(
                np.linspace(x_start + (X_SPACING/2), x_end + (X_SPACING/2), int((x_end - x_start) / X_SPACING) + 1, endpoint=False), 
                np.linspace(y_start + (Y_SPACING/2), y_end + (Y_SPACING/2), int((y_end - y_start) / Y_SPACING) + 1, endpoint=False)
                ))

    geometry = [Point(lon, lat).buffer(RADIUS) for lon, lat in lon_lat]

    center_x = [lon for lon, _ in lon_lat]
    center_y = [lat for _, lat in lon_lat]

    df = gpd.GeoDataFrame({'center_x':center_x, 'center_y':center_y}, geometry = geometry, crs=CRS.from_epsg(4326))
    
    return df

In [7]:
# an arbitrary AOI in CA for testing purposes
test_aoi = Polygon([[-116, 40],[-119, 40],[-119, 38],[-116, 38],[-116, 40]])
df = make_grid(*test_aoi.bounds)
df.to_file(output_path/'test_aoi_df')


In [8]:
def get_area_coverage(aoi: Polygon, search_dates: List[datetime], x_res = 1, y_res = 0.25, radius = 0.1):
    
    x_start, y_start, x_end, y_end = aoi.bounds
    
    df = make_grid(x_start, y_start, x_end, y_end, x_res, y_res, radius)
    
    indices = df.geometry.intersects(aoi)
    df_points = df[indices].reset_index(drop=True)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(get_cadence_df, tqdm(df_points.geometry), [search_dates]*len(df_points)))
        
    df_temp = df_points.copy()
    df_temp['av_cad_hours'] = [i.cadence.mean().seconds/3600 for i in results]
    df_temp['avail_sensors'] = [i.sensor.unique() for i in results]
    df_temp['n_avail_sensors'] = [len(i.sensor.unique()) for i in results]
    
    # Re-grid results to 1x1 degree cells
    df_temp['lon'] = df_temp.geometry.map(get_lon)
    df_temp['lat'] = df_temp.geometry.map(get_lat)
    df_ag = df_temp.groupby(['lon', 'lat']).mean().reset_index()
    geom = df_ag.apply(make_box, axis=1)
    
    df_ag = gpd.GeoDataFrame(df_ag, geometry=geom, crs = CRS.from_epsg(4326))
    
    return df_ag

In [9]:
df = get_area_coverage(test_aoi,[datetime(2022,1,1),datetime(2022,2,1)])

100%|██████████| 36/36 [00:00<00:00, 365.66it/s]
/Users/karthikv/opt/anaconda3/envs/opera_coverage/lib/python3.10/site-packages/opera_coverage/formatting.py:32: UserWarning: Dataframe is empty! Check inputs.
  warn('Dataframe is empty! Check inputs.')
/Users/karthikv/opt/anaconda3/envs/opera_coverage/lib/python3.10/site-packages/opera_coverage/formatting.py:32: UserWarning: Dataframe is empty! Check inputs.
  warn('Dataframe is empty! Check inputs.')


In [10]:
df.to_file(output_path/'test_coverage_df')

/var/folders/0j/_hzqdqys6wg5g5khfdl7tdk00000gr/T/ipykernel_37983/1742401870.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(output_path/'test_coverage_df')


In [11]:
# need the updated shape2gdf method for this to work
_ = shape2gdf(test_aoi, filename = 'test_aoi_gdf', to_file = True)

TypeError: shape2gdf() got an unexpected keyword argument 'to_file'